In [20]:
import datetime
from ib_insync import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [2]:
ib = connect_to_IB(clientid=1)

#### Initiate PortfolioManager Class

In [3]:
pm = PortfolioManager(ib)

In [5]:
# Get current portfolio
df_ib = pm.get_positions_from_ib()

### Connect to ArcticDB

In [14]:
ac = initialize_db('db')
print(ac.list_libraries())
# general = ac.get_library('general')
portfolio_lib = ac.get_library('portfolio')
portfolio_lib.list_symbols()

['general', 'portfolio', 'spx500', 'stocks']


['positions']

In [15]:
df_ac = portfolio_lib.read('positions').data
df_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,ARM,,Call 150.0 20240216,2024-02-11 22:54,-1.0,-0.065282,177.198900,0.730747,58.761172,"Option(conId=682880857, symbol='ARM', lastTrad...",...,2024-02-11,None,False,-73.070000,104.120000,USD,0.0,U7706434,-67.795509,1.0778
1,BABA,,STK,2024-02-11 22:54,30.0,1.927367,76.181877,71.910004,-5.607466,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-11,None,False,2157.300000,-128.160000,USD,0.0,U7706434,2001.577287,1.0778
2,EWT,,STK,2024-02-11 22:54,200.0,8.342724,44.256828,46.689999,5.497841,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-11,None,False,9338.000000,486.630000,USD,0.0,U7706434,8663.945073,1.0778
3,EWT,,Put 44.26 20240216,2024-02-11 22:54,-1.0,-0.001036,39.700000,0.011609,97.075705,"Option(conId=671913111, symbol='EWT', lastTrad...",...,2024-02-11,None,False,-1.160000,38.540000,USD,0.0,U7706434,-1.076266,1.0778
4,GOOGL,,STK,2024-02-11 22:54,30.0,3.992232,146.106477,148.949997,1.946197,"Stock(conId=208813719, symbol='GOOGL', right='...",...,2024-02-11,None,False,4468.500000,85.310000,USD,0.0,U7706434,4145.945444,1.0778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,TLT,,Put 97.0 20240216,2024-02-13 22:24,-1.0,-0.002829,156.377100,4.620003,-195.439853,"Option(conId=614854999, symbol='TLT', lastTrad...",...,2024-02-11,None,False,-4.620003,-151.757097,USD,0.0,U7706434,-2.908648,1.0778
216,TLT,,Call 98.0 20240216,2024-02-13 22:24,-1.0,-0.000027,73.369800,0.002668,99.636315,"Option(conId=614854493, symbol='TLT', lastTrad...",...,2024-02-11,None,False,-0.002668,-73.367132,USD,0.0,U7706434,-0.027881,1.0778
217,TLT,,Put 93.0 20240223,2024-02-13 22:24,-1.0,-0.000460,93.959600,1.147836,-22.162690,"Option(conId=676006202, symbol='TLT', lastTrad...",...,2024-02-11,None,False,-1.147836,-92.811764,USD,0.0,U7706434,-0.472774,1.0778
218,TSLA,,STK,2024-02-13 22:24,-20.0,-3.370604,233.983450,184.160004,21.293577,"Stock(conId=76792991, symbol='TSLA', right='0'...",...,2024-02-11,None,False,-3683.200073,-49.823446,USD,0.0,U7706434,-3465.280345,1.0778


#### Get latest timestamp from arcticDB df

In [53]:
df_ac = portfolio_lib.read('positions').data
filtered_df = df_ac[df_ac['strategy'] == "SVIX"]
filtered_df

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
19,VXM,SVIX,VXMJ4 20240417,2024-02-11 22:54,-3.0,-4.181189,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-4680.000000,166.570000,USD,0.0,U7706434,-4342.178512,1.0778
39,VXM,SVIX,VXMJ4 20240417,2024-02-11 22:55,-3.0,-4.181189,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-4342.178512,1.0778
59,VXM,SVIX,VXMJ4 20240417,2024-02-11 22:57,-3.0,-0.041812,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-43.421786,1.0778
79,VXM,SVIX,VXMJ4 20240417,2024-02-11 22:59,-3.0,-0.041812,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-43.421786,1.0778
99,VXM,SVIX,VXMJ4 20240417,2024-02-11 23:00,-3.0,-0.041812,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-43.421786,1.0778
119,VXM,SVIX,VXMJ4 20240417,2024-02-11 23:01,-3.0,-0.041812,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-43.421786,1.0778
139,VXM,SVIX,VXMJ4 20240417,2024-02-11 23:02,-3.0,-0.041812,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-43.421786,1.0778
159,VXM,SVIX,VXMJ4 20240417,2024-02-11 23:03,-3.0,-0.041812,1615.525,15.600000,3.436961,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.800001,-1599.925000,USD,0.0,U7706434,-43.421786,1.0778
179,VXM,SVIX,VXMJ4 20240417,2024-02-12 16:11,-3.0,NaN,1615.525,15.568553,3.631619,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-46.705659,-1599.956447,USD,0.0,U7706434,NaN,1.0778
199,VXM,SVIX,VXMJ4 20240417,2024-02-13 12:09,-3.0,-0.041984,1615.525,15.953000,1.251915,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-11,None,False,-47.859000,-1599.572000,USD,0.0,U7706434,-43.358391,1.0778


In [50]:
df_merged = pm.match_ib_positions_with_arcticdb()
df_merged

Msg from function 'match_ib_positions_with_arcticdb': No Symbol 'positions' in library 'portfolio'
Creating an arcticdb entry 'positions' in library 'portfolio'


[2024-02-13 23:42:52.164] [arcticdb] [info] Column close_dt does not have non null elements.


,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,2024-02-13 23:42,ARM,Call 150.0 20240216,-1.0,-0.163289,177.198900,1.807977,-2.030921,,"Option(conId=682880857, symbol='ARM', lastTrad...",...,2024-02-13,None,False,-180.80,-3.60,USD,0.0,U7706434,-167.842555,1.0772
1,2024-02-13 23:42,BABA,STK,30.0,1.940235,76.181877,71.610001,-6.001265,,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-13,None,False,2148.30,-137.16,USD,0.0,U7706434,1994.337170,1.0772
2,2024-02-13 23:42,EWT,STK,200.0,8.247783,44.256828,45.661243,3.173329,,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-13,None,False,9132.25,280.88,USD,0.0,U7706434,8477.766431,1.0772
3,2024-02-13 23:42,EWT,Put 44.26 20240216,-1.0,-0.006060,39.700000,0.067099,83.098388,,"Option(conId=671913111, symbol='EWT', lastTrad...",...,2024-02-13,None,False,-6.71,32.99,USD,0.0,U7706434,-6.229113,1.0772
4,2024-02-13 23:42,IAU,STK,300.0,10.214616,38.454255,37.700001,-1.961432,,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-02-13,None,False,11310.00,-226.28,USD,0.0,U7706434,10499.443000,1.0772
5,2024-02-13 23:42,IAU,Call 39.0 20240216,-2.0,-0.000036,23.350200,0.000222,99.904797,,"Option(conId=671974250, symbol='IAU', lastTrad...",...,2024-02-13,None,False,-0.04,46.66,USD,0.0,U7706434,-0.037133,1.0772
6,2024-02-13 23:42,INDA,STK,100.0,4.515023,49.712499,49.991951,0.562136,,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-02-13,None,False,4999.20,27.95,USD,0.0,U7706434,4640.920906,1.0772
7,2024-02-13 23:42,IUSQ,STK,100.0,6.982303,65.602785,71.769997,9.400838,,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-02-13,None,False,7177.00,616.72,EUR,0.0,U7706434,7177.000000,1.0000
8,2024-02-13 23:42,IWM,STK,100.0,17.615922,199.616920,195.050003,-2.287841,,"Stock(conId=9579970, symbol='IWM', right='0', ...",...,2024-02-13,None,False,19505.00,-456.69,USD,0.0,U7706434,18107.129595,1.0772
9,2024-02-13 23:42,IWM,Call 199.0 20240216,-1.0,-0.070879,183.368900,0.784831,57.199356,,"Option(conId=637114226, symbol='IWM', lastTrad...",...,2024-02-13,None,False,-78.48,104.89,USD,0.0,U7706434,-72.855551,1.0772


In [49]:
df_ac = portfolio_lib.read('positions').data
last_two_timestamps = df_ac['timestamp'].unique()[-2:]

filtered_df = df_ac[df_ac['timestamp'].isin(last_two_timestamps)]
latest_positions = filtered_df.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,ARM,,Call 150.0 20240216,2024-02-13 22:24,-1.0,-0.008946,177.198900,1.452623,18.023029,"Option(conId=682880857, symbol='ARM', lastTrad...",...,2024-02-11,None,False,-1.452623,-175.746277,USD,0.0,U7706434,-9.197384,1.0778
1,BABA,,STK,2024-02-13 22:24,30.0,1.978045,76.181877,71.580002,-6.040642,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-11,None,False,2147.400056,-4.601875,USD,0.0,U7706434,2033.605551,1.0778
2,EWT,,Put 44.26 20240216,2024-02-13 22:24,-1.0,-0.000013,39.700000,0.067099,83.098388,"Option(conId=671913111, symbol='EWT', lastTrad...",...,2024-02-11,None,False,-0.067099,-39.632901,USD,0.0,U7706434,-0.013213,1.0778
3,EWT,,STK,2024-02-13 22:24,200.0,8.445048,44.256828,45.640278,3.125957,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-11,None,False,9128.055570,1.383449,USD,0.0,U7706434,8682.258058,1.0778
4,GOOGL,,STK,2024-02-13 12:09,30.0,3.993642,146.106477,146.330002,0.152988,"Stock(conId=208813719, symbol='GOOGL', right='...",...,2024-02-11,None,False,4389.900056,0.223525,USD,0.0,U7706434,4124.414987,1.0778
5,IAU,,Call 39.0 20240216,2024-02-13 22:24,-2.0,-0.000046,23.350200,0.000222,99.904797,"Option(conId=671974250, symbol='IAU', lastTrad...",...,2024-02-11,None,False,-0.000445,-23.349978,USD,0.0,U7706434,-0.046849,1.0778
6,IAU,,STK,2024-02-13 22:24,300.0,10.385955,38.454255,37.701225,-1.958247,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-02-11,None,False,11310.367590,-0.753029,USD,0.0,U7706434,10677.682923,1.0778
7,INDA,,STK,2024-02-13 22:24,100.0,4.551869,49.712499,49.981346,0.540804,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-02-11,None,False,4998.134615,0.268847,USD,0.0,U7706434,4679.725200,1.0778
8,IUSQ,,STK,2024-02-13 22:24,100.0,7.033092,65.602785,71.769997,9.400838,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-02-11,None,False,7176.999665,6.167212,EUR,0.0,U7706434,7230.642700,1.0000
9,IWM,,Call 199.0 20240216,2024-02-13 22:24,-1.0,-0.003977,183.368900,0.695072,62.094352,"Option(conId=637114226, symbol='IWM', lastTrad...",...,2024-02-11,None,False,-0.695072,-182.673828,USD,0.0,U7706434,-4.088675,1.0778


# Add some data to portfolio

In [19]:
dummy_data = df.loc[0,:].copy()
dummy_data['position'] = dummy_data['position'] / 4
dummy_data['% of nav'] = dummy_data['% of nav'] / 4
dummy_data['averageCost'] = dummy_data['averageCost'] * 1.1
dummy_data['unrealizedPNL'] = (dummy_data['marketPrice'] - dummy_data['averageCost']) * dummy_data['position']
dummy_data['marketValue'] = dummy_data['marketPrice'] * dummy_data['position']
dummy_data['strategy'] = "buy & hold"
if dummy_data['asset class'] == 'STK' or dummy_data['asset class'] == 'FUT':
    pnl = ((dummy_data.marketPrice/(dummy_data.averageCost)) -1)
    pnl = pnl *(-1) if dummy_data.position < 0 else pnl
else:
    if dummy_data.position < 0:
        pnl = ((dummy_data.marketPrice/(dummy_data.averageCost/100)) -1) * (-1)
    else:
        pnl = ( (dummy_data.marketPrice/ (dummy_data.averageCost/100)) -1)
dummy_data['pnl %'] = pnl
dummy_data = pd.DataFrame(dummy_data).T


In [20]:
dummy_data

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,2024-02-02 18:35,EWT,STK,50.0,0.020126,48.682511,44.716,-0.081477,buy & hold,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-02,2235.79998,-198.325587,USD,0.0,U7706434,8288.991357,1.078925


In [25]:
# Convert 'contract' column to string as ArcticDB may not handle custom objects well
dummy_data['contract'] = dummy_data['contract'].astype(str)
dummy_data['trade'] = dummy_data['trade'].astype(str)
# Ensure that all other columns are of a consistent and compatible data type
# For example, convert all numeric columns to float

numeric_columns = ['position', '% of nav', 'averageCost', 'marketPrice', 'pnl %', 'marketValue', 'unrealizedPNL', 'realizedPNL','marketValue_base','fx_rate']
for col in numeric_columns:
    dummy_data[col] = pd.to_numeric(dummy_data[col], errors='coerce')

# Now write the modified DataFrame to ArcticDB
portfolio_lib.write('positions', dummy_data)


VersionedItem(symbol='positions', library='portfolio', data=n/a, version=1, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

# Compare df_ac & df_ib

In [16]:
portfolio_lib.read('positions').version

7

In [43]:
df_ac = portfolio_lib.read('positions').data
df_ac



NoSuchVersionException: E_NO_SUCH_VERSION read_dataframe_version: version matching query 'latest' not found for symbol 'positions'

#### Now, we will get ib positions and match them with arcticdb entries

In [7]:
df_ib = pm.get_positions_from_ib()
df_ib.head()

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,2024-02-11 21:52,ARM,Call 150.0 20240216,-1.0,-0.065282,177.198900,0.730747,58.761172,,"Option(conId=682880857, symbol='ARM', lastTrad...",...,2024-02-11,None,False,-73.07,104.12,USD,0.0,U7706434,-67.795509,1.0778
1,2024-02-11 21:52,BABA,STK,30.0,1.927367,76.181877,71.910004,-5.607466,,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-11,None,False,2157.30,-128.16,USD,0.0,U7706434,2001.577287,1.0778
2,2024-02-11 21:52,EWT,STK,200.0,8.342724,44.256828,46.689999,5.497841,,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-11,None,False,9338.00,486.63,USD,0.0,U7706434,8663.945073,1.0778
3,2024-02-11 21:52,EWT,Put 44.26 20240216,-1.0,-0.001036,39.700000,0.011609,97.075705,,"Option(conId=671913111, symbol='EWT', lastTrad...",...,2024-02-11,None,False,-1.16,38.54,USD,0.0,U7706434,-1.076266,1.0778
4,2024-02-11 21:52,GOOGL,STK,30.0,3.992232,146.106477,148.949997,1.946197,,"Stock(conId=208813719, symbol='GOOGL', right='...",...,2024-02-11,None,False,4468.50,85.31,USD,0.0,U7706434,4145.945444,1.0778


In [18]:
# Assuming df_ac is from ArcticDB and df_ib is the incoming DataFrame you want to append

print("ArcticDB DataFrame dtypes:")
print(df_ac.dtypes)

print("\nIncoming DataFrame dtypes:")
print(df_ib.dtypes)

# Comparing data types column by column
for col in df_ac.columns:
    ac_dtype = df_ac[col].dtype
    ib_dtype = df_ib[col].dtype
    print(f"Column '{col}': ArcticDB dtype={ac_dtype}, Incoming dtype={ib_dtype}, Match={ac_dtype == ib_dtype}")


ArcticDB DataFrame dtypes:
timestamp            object
symbol               object
asset class          object
position            float64
% of nav            float64
averageCost         float64
marketPrice         float64
pnl %               float64
strategy             object
contract             object
trade                object
open_dt              object
close_dt             object
deleted                bool
marketValue         float64
unrealizedPNL       float64
currency             object
realizedPNL         float64
account              object
marketValue_base    float64
fx_rate             float64
dtype: object

Incoming DataFrame dtypes:
timestamp            object
symbol               object
asset class          object
position            float64
% of nav            float64
averageCost         float64
marketPrice         float64
pnl %               float64
strategy             object
contract             object
trade                object
open_dt              object
close_d

In [32]:
df_ib['contract'] = df_ib['contract'].astype(str)
df_ib['trade'] = df_ib['trade'].astype(str)
portfolio_lib.append('positions',df_ib, validate_index=False)

[2024-02-11 22:19:08.424] [arcticdb] [info] Column close_dt does not have non null elements.


VersionedItem(symbol='positions', library='portfolio', data=n/a, version=10, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [41]:
from arcticdb import Arctic, LibraryOptions

# Create LibraryOptions with dynamic_schema enabled
library_options = LibraryOptions(dynamic_schema=True)

# Create the library with the specified options
ac.create_library('portfolio', library_options=library_options)


Library(Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)), path=portfolio, storage=s3_storage)

In [40]:
ac.create_library('portfolio',library_options=['dynamic_schema'==True])

AttributeError: 'list' object has no attribute 'encoding_version'

In [42]:
portfolio_lib.delete('positions')

ac.delete_library('portfolio')

[2024-02-11 22:54:38.094] [arcticdb] [warning] Symbol: positions does not exist.


In [10]:
def match_ib_positions_with_arcticdb():
    library = ac.get_library('portfolio',create_if_missing=True)
    df_ac = library.read('positions').data
    latest_positions = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
    df_ib =  df_ib # self.get_positions_from_ib()

In [12]:
df_ac = portfolio_lib.read('positions').data
latest_positions_in_ac = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions_in_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,EWT,buy & hold,STK,2024-02-02 18:35,50.0,0.020126,48.682511,44.716,-0.081477,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-02,2235.79998,-198.325587,USD,0.0,U7706434,8288.991357,1.078925


In [39]:
df_merged = pd.DataFrame()
total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    # total_average_cost = row['averageCost']
    # total_market_price = row['marketPrice']

    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & 
                                           (latest_positions_in_ac['asset class'] == asset_class)]
    
    if not strategy_entries_in_ac.empty:
        strategy_entry = strategy_entries_in_ac.iloc[0]
        if total_position == strategy_entry.position:
            # Since the positions match, we use the latest data from ib and only add the strategy field
            row['strategy'] = strategy_entry.strategy
            df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)
        else:
            for entry in strategy_entries_in_ac:
                # update fields in: ['marketPrice', '% of nav','pnl %']
                strategy_entries_in_ac.loc[entry,'% of nav'] = ((strategy_entries_in_ac[entry,'position'] / total_position) * row['marketValue']) / total_equity
                # other update fields here

                # add entry to df_merged
            # ---------------------------------------------
                    
            # Calculate the averageCost, '% of nav','pnl %' and position for the residual
            assigned_position = strategy_entries_in_ac['position'].sum()
            residual_position = total_position - assigned_position

            # Calculate the average cost for the residual position
            ib_avg_cost = row['averageCost']
            ac_avg_cost = strategy_entries_in_ac.apply(lambda x: x['averageCost'] * (x['position'] / total_position), axis=1).sum()
            residual_avg_cost = (ib_avg_cost - ac_avg_cost) * (total_position / residual_position)
            #print(f"{symbol:} ib_avg_cost: {ib_avg_cost}, residual avg cost {residual_avg_cost}, ac_avg_cost {strategy_entries_in_ac.averageCost}")

            # do other calculations

            # concat residual position to df_merged
            
    else:
    # we have positions that are not in arcticdb that we should display & save as well
        pass
        

df_merged
    

EWT ib_avg_cost: 44.2568285, residual avg cost 48.68251135, ac_avg_cost 0    39.831146
Name: averageCost, dtype: float64


""


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wu

In [54]:
df_merged = pd.DataFrame()
total_equity = sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

# Iterate through the positions obtained from Interactive Brokers
for index, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    
    # Filter the ArcticDB DataFrame for entries with the same symbol and asset class
    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]
    
    if not strategy_entries_in_ac.empty:
        # Calculate the total position assigned to strategies in ArcticDB
        assigned_position = strategy_entries_in_ac['position'].sum()
        
        # If the total position from IB matches the sum of assigned positions in ArcticDB
        if total_position == assigned_position:
            # Use the row data and add the strategy info
            row['strategy'] = strategy_entries_in_ac['strategy'].values[0]  # Assuming one strategy per symbol/asset class
            df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)
        else:
            # There's a discrepancy between the positions, so we need to handle the residual
            # First, update the existing entries from ArcticDB
            strategy_entries_in_ac['marketPrice'] = row['marketPrice']
            strategy_entries_in_ac['% of nav'] = (strategy_entries_in_ac['position'] * row['marketPrice']) / total_equity
            strategy_entries_in_ac['pnl %'] = (strategy_entries_in_ac['marketPrice'] / strategy_entries_in_ac['averageCost'] - 1) * 100
            
            # Append the updated entries to df_merged
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # Now, calculate and append the residual position entry
            residual_position = total_position - assigned_position
            residual_avg_cost = ((row['averageCost'] * total_position) - (strategy_entries_in_ac['averageCost'] * assigned_position)) / residual_position
            residual_avg_cost = residual_avg_cost[0]
            residual_market_value = residual_position * row['marketPrice']
            residual_percent_nav = residual_market_value / total_equity
            residual_pnl_percent = ((row['marketPrice'] / residual_avg_cost) - 1) * 100
            
            residual_entry = row.copy()
            residual_entry['position'] = residual_position
            residual_entry['averageCost'] = residual_avg_cost
            residual_entry['marketValue'] = residual_market_value
            residual_entry['% of nav'] = residual_percent_nav
            residual_entry['pnl %'] = residual_pnl_percent
            residual_entry['strategy'] = ''  # No strategy assigned yet
            
            df_merged = pd.concat([df_merged, pd.DataFrame([residual_entry])], ignore_index=True)
            
    else:
        # If there are no entries in ArcticDB for this symbol/asset class, just append the row from ib
        df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

df_merged


,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,100.0,0.044055,39.831146,45.020000,13.027129,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.070100,437.955535,0.0,U7706434
1,EWT,,STK,2024-01-26 21:23,100.0,0.044055,48.682511,45.020000,-7.523258,"Stock(conId=253190576, symbol='EWT', right='0'...",4502.000045,152.630000,0.0,U7706434
2,IAU,,STK,2024-01-26 21:23,300.0,0.112103,38.454255,38.183998,-0.007028,"Stock(conId=474829650, symbol='IAU', right='0'...",11455.200000,-81.080000,0.0,U7706434
3,IAU,,Call 39.0 20240216,2024-01-26 21:23,-2.0,-0.000324,23.350200,0.165493,0.291256,"Option(conId=671974250, symbol='IAU', lastTrad...",-33.100000,13.600000,0.0,U7706434
4,INDA,,STK,2024-01-26 21:23,100.0,0.048095,49.712499,49.146000,-0.011396,"Stock(conId=101484826, symbol='INDA', right='0...",4914.600000,-56.650000,0.0,U7706434
5,IUSQ,,STK,2024-01-26 21:23,100.0,0.068934,65.602785,70.440002,0.073735,"Stock(conId=239363241, symbol='IUSQ', right='0...",7044.000000,483.720000,0.0,U7706434
6,IWM,,STK,2024-01-26 21:23,100.0,0.191716,199.616920,195.903992,-0.018600,"Stock(conId=9579970, symbol='IWM', right='0', ...",19590.400000,-371.290000,0.0,U7706434
7,IWM,,Call 199.0 20240216,2024-01-26 21:23,-1.0,-0.002821,183.368900,2.883058,-0.572272,"Option(conId=637114226, symbol='IWM', lastTrad...",-288.310000,-104.940000,0.0,U7706434
8,PRX,,STK,2024-01-26 21:23,250.0,0.069140,26.561554,28.260000,0.063944,"Stock(conId=382625193, symbol='PRX', right='0'...",7065.000000,424.610000,0.0,U7706434
9,SOXX,,Put 525.0 20240216,2024-01-26 21:23,-1.0,-0.000827,699.944800,0.844985,0.879278,"Option(conId=653437229, symbol='SOXX', lastTra...",-84.500000,615.450000,0.0,U7706434


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde

In [42]:
# helper functions
def update_data(output_df,row):
    output_df['marketPrice'] = row['marketPrice']
    output_df['marketValue_base'] = output_df['marketValue'] / row['fx_rate']
    output_df['pnl %'] = calculate_pnl(row.marketPrice,row.averageCost,row.position,row.contract)

    output_df['unrealizedPNL'] = output_df['marketPrice'] - output_df['averageCost']
    output_df['marketValue'] = (output_df['marketPrice'] * output_df['position']) 
    output_df['% of nav'] = (output_df['marketValue_base'] / total_equity) * 100
    return output_df

def calculate_pnl(market_price, average_cost, position, contract=None):
    """
    Calculate PNL percentage based on market price, average cost, and position.
    For options and futures, contract details are considered for multiplier effect.

    Parameters:
    - market_price: The current market price of the asset.
    - average_cost: The average cost of the asset.
    - position: The quantity of the asset.
    - contract: The contract object containing details like type and multiplier.

    Returns:
    - pnl_percent: The PNL percentage.
    """
    pnl_percent = 0
    if contract is not None:
        if isinstance(contract, Stock):
            pnl = ((market_price / average_cost) - 1)
        elif isinstance(contract, Option) or isinstance(contract, Future):
            multiplier = 100 if isinstance(contract, Option) else float(contract.multiplier)
            pnl = ((market_price / (average_cost / multiplier)) - 1)
    else:
        pnl = ((market_price / average_cost) - 1)

    pnl_percent = pnl * (-1) if position < 0 else pnl
    return pnl_percent * 100

                                                                    
def handle_residual(strategy_entries_in_ac, row):
    # Calculate the residual position
    total_position = row['position']
    assigned_position_sum = strategy_entries_in_ac['position'].sum()
    residual_position = total_position - assigned_position_sum

    # No residual to handle
    if residual_position == 0:
        return pd.DataFrame()

    # Calculate the weighted average cost for assigned positions
    weighted_avg_costs = (strategy_entries_in_ac['averageCost'] * strategy_entries_in_ac['position']).sum()
    total_weighted_cost = row['averageCost'] * total_position

    # Calculate the average cost for the residual position
    residual_avg_cost = (total_weighted_cost - weighted_avg_costs) / residual_position

    # Calculate updated market values for the residual
    market_price = row['marketPrice']
    residual_market_value = residual_position * market_price
    residual_percent_nav = ((residual_market_value / row.fx_rate) / total_equity) * 100
    pnl_percent = calculate_pnl(market_price,residual_avg_cost,residual_position,row.contract)

    # timestamp	symbol	asset class	position	% of nav	averageCost	marketPrice	pnl %	strategy	contract	trade	open_dt	marketValue	unrealizedPNL	currency	realizedPNL	account	marketValue_base	fx_rate
#  0	2024-02-03 02:16	EWT	STK	200.0	8.025971	44.256828	44.758621	1.133820		Stock(conId=253190576, symbol='EWT', right='0'...	None	2024-02-03	8951.72	100.36	USD	0.0	U7706434	8233.738043	1.0872
         
    # Prepare the residual row data
    residual_row = {
        'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
        'symbol': row['symbol'],
        'asset class': row['asset class'],
        'position': residual_position,
        '% of nav': residual_percent_nav,
        'averageCost': residual_avg_cost,
        'marketPrice': market_price,
        'pnl %': pnl_percent,
        'strategy': '',  # This can be updated to assign a strategy later
        'contract': row['contract'],
        'trade': None,
        'open_dt': datetime.date.today().isoformat(),
        'marketValue': residual_market_value,
        'unrealizedPNL': (market_price - residual_avg_cost) * residual_position,
        'currency': row.currency,
        'realizedPNL': 0,  # Assuming no realized P&L for the residual; update as needed
        'account': row['account'],
        'marketValue_base': residual_market_value / row.fx_rate,
        'fx_rate': row.fx_rate}

    # Return the residual row as a DataFrame
    return pd.DataFrame([residual_row])


    
# def calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position):
#     # Implement logic to calculate the average cost for the residual position
#     return ((total_average_cost * total_position) - (assigned_average_cost * assigned_position)) / (total_position - assigned_position)


In [43]:
df_merged = pd.DataFrame()
# total_equity = sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

# Iterate through the positions obtained from Interactive Brokers
for index, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']

    # Filter the ArcticDB DataFrame for entries with the same symbol and asset class
    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]
    sum_of_position_entries = strategy_entries_in_ac['position'].sum()

    if strategy_entries_in_ac.empty: # no database entry, add position
        df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

    elif len(strategy_entries_in_ac) == 1:
        if total_position == sum_of_position_entries:
            # the positions match, we should only update fields relevant for marketdata in ac
            strategy_entries_in_ac = update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
        else:
            #update marketdata relevant fields for the existing db entry
            strategy_entries_in_ac = update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # handle the residual and concat to df_merged
            residual = handle_residual(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, residual], ignore_index=True)

    elif len(strategy_entries_in_ac) > 1:
        if total_position == sum_of_position_entries:
            strategy_entries_in_ac = update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
        else:
            #update marketdata relevant fields for the existing db entry
            strategy_entries_in_ac = update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # handle the residual and concat to df_merged
            residual = handle_residual(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, residual], ignore_index=True)

df_merged         

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,EWT,buy & hold,STK,2024-02-02 18:35,50.0,2.004583,48.682511,44.758621,1.133820,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-02,2237.93106,-3.923890,USD,0.0,U7706434,2056.475331,1.078925
1,EWT,,STK,2024-02-03 23:30,150.0,6.019481,42.781601,44.758621,4.621193,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-03,6713.79318,296.553047,USD,0.0,U7706434,6175.306457,1.087200
2,EWT,,Put 44.26 20240216,2024-02-03 02:16,-1.0,-0.034330,39.700040,0.382858,3.562377,"Option(conId=671913111, symbol='EWT', lastTrad...",None,2024-02-03,-38.29000,1.410000,USD,0.0,U7706434,-35.218911,1.087200
3,IAU,,STK,2024-02-03 02:16,300.0,10.379782,38.454255,38.590115,0.353303,"Stock(conId=474829650, symbol='IAU', right='0'...",None,2024-02-03,11577.03000,40.760000,USD,0.0,U7706434,10648.482340,1.087200
4,IAU,,Call 39.0 20240216,2024-02-03 02:16,-2.0,-0.040615,23.350200,0.226483,3.006141,"Option(conId=671974250, symbol='IAU', lastTrad...",None,2024-02-03,-45.30000,1.400000,USD,0.0,U7706434,-41.666667,1.087200
5,INDA,,STK,2024-02-03 02:16,100.0,4.517117,49.712499,50.381367,1.345472,"Stock(conId=101484826, symbol='INDA', right='0...",None,2024-02-03,5038.14000,66.890000,USD,0.0,U7706434,4634.050773,1.087200
6,IUSQ,,STK,2024-02-03 02:16,100.0,6.962756,65.602785,71.430000,8.882573,"Stock(conId=239363241, symbol='IUSQ', right='0...",None,2024-02-03,7143.00000,582.720000,EUR,0.0,U7706434,7143.000000,1.000000
7,IWM,,STK,2024-02-03 02:16,100.0,17.394630,199.616920,194.009995,-2.808843,"Stock(conId=9579970, symbol='IWM', right='0', ...",None,2024-02-03,19401.00000,-560.690000,USD,0.0,U7706434,17844.922737,1.087200
8,IWM,,Call 199.0 20240216,2024-02-03 02:16,-1.0,-0.137115,183.368900,1.529287,16.600506,"Option(conId=637114226, symbol='IWM', lastTrad...",None,2024-02-03,-152.93000,30.440000,USD,0.0,U7706434,-140.664091,1.087200
9,PRX,,STK,2024-02-03 02:16,250.0,6.872103,26.561554,28.200001,6.168488,"Stock(conId=382625193, symbol='PRX', right='0'...",None,2024-02-03,7050.00000,409.610000,EUR,0.0,U7706434,7050.000000,1.000000


In [33]:
import pandas as pd

df_merged = pd.DataFrame()  # Ensure df_merged is initialized as an empty DataFrame

# ... your code ...

for index, row in df_ib.iterrows():
    # ... your code ...
    df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

# ... your code ...


In [35]:
df_merged.append(row)

AttributeError: 'DataFrame' object has no attribute 'append'

In [22]:
# Assuming df_ib and latest_positions_in_ac are your DataFrames from IB and ArcticDB

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    # total_average_cost = row['averageCost']
    # total_market_price = row['marketPrice']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & 
                                           (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        assigned_position = strategy_rows['position'].sum()
        residual_position = total_position - assigned_position

        if len(strategy_rows) == 1 and total_position == strategy_rows.iloc[0]['position']:
            # Case 1: Single strategy row and matching position
            # Update all columns in ArcticDB for this symbol & asset class combination except 'strategy'
            # Example: Update marketPrice and other related fields in ArcticDB
            pass

        else:
            # Case 2: Single/Multiple strategy rows and differing position
            # Calculate averageCost for the residual position
            assigned_average_cost = strategy_rows['averageCost'].iloc[0] if len(strategy_rows) == 1 else calculate_weighted_average_cost(strategy_rows)
            residual_average_cost = calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position)

            # Update marketPrice, marketValue, % of NAV, pnl%, unrealizedPNL for the assigned positions in ArcticDB
            # And calculate the corresponding values for the residual position
            pass

def calculate_weighted_average_cost(strategy_rows):
    # Implement logic to calculate a weighted average cost for multiple strategy rows
    pass

def calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position):
    # Implement logic to calculate the average cost for the residual position
    return ((total_average_cost * total_position) - (assigned_average_cost * assigned_position)) / (total_position - assigned_position)

# Further implement the functions or logic needed in 'pass' sections

1


In [57]:
for _, row in df_ib.iterrows():
    symbol = row.symbol
    asset_class = row['asset class']
    total_position = row['position']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        if len(strategy_rows) == 1 and total_position == strategy_rows.position:
            # update all cols in ac for this symbol & asset class combination except col strategies
            pass
        if len(strategy_rows) == 1 and total_position != strategy_rows.position:
            # 1. update cols marketPrice in ac from ib query and calculate new values for 'marketValue' (position*marketPrice), '% of nav', 'pnl%','unrealized'
            # calculate averageCost for the residual position (ib total averageCost = ac.averageCost * (ac.position/total_position) + residual.averageCost * (residual.position/total_position))
            # with the averageCost value and the position for the residual we can calculate 'marketValue', '% of nav', 'pnl%','unrealized'

            # if possible generalize the above for the cases where len(strategy_rows) > 1
            pass


1
EWT in ac
100.0 100.0


In [42]:
latest_positions['symbol'].values

array(['EWT'], dtype=object)